In [1]:
## Import libraries and csv

import pandas as pd

py_df = pd.read_csv('purchase_data.csv',encoding='utf8')
py_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
## Create a new dataframe with duplicates flagged. 
## This is to handle the gender analysis. 

dups = py_df.duplicated(subset='SN',keep='first')
py_df['dups'] = dups

In [3]:
## Find unique player count

dups = py_df.duplicated(subset='SN',keep='first')
py_df['dups'] = dups
dup_df = py_df[py_df['dups']==False]
unique_p_d = dup_df.count()
print('There are ' + str(unique_p_d[0]) + ' unique players.')

There are 576 unique players.


In [4]:
# Purchasing Analysis (Total)

# Number of Unique Items
# Average Purchase Price
# Total Number of Purchases
# Total Revenue

unique_items = py_df['Item ID'].unique()
unique_count = int(len(unique_items))
avg = float(py_df['Price'].mean())
tot_purchase = int(py_df['Purchase ID'].count())
rev = float(py_df['Price'].sum())

pa_dict = {'Number of Unique Items': unique_count,
          'Average Purchasing Price': round(avg,2),
          'Total Number of Purchases' : tot_purchase,
          'Total Revenue' : round(rev,2)}

pa_df = pd.Series(pa_dict)
pa_df

Number of Unique Items        183.00
Average Purchasing Price        3.05
Total Number of Purchases     780.00
Total Revenue                2379.77
dtype: float64

In [5]:
# Gender Demographics

# Percentage and Count of Male Players
# Percentage and Count of Female Players
# Percentage and Count of Other / Non-Disclosed

gender_df = dup_df['Gender']
unique_gender = gender_df.unique()

total_count = gender_df.count()
count_arr = []
percent_arr = []
store_g = []

for i in range(len(unique_gender)):
    g = unique_gender[i]
    store_g.append(g)
    counter = gender_df[gender_df==g].count()
    percentage = float(round((counter/total_count),2))
    count_arr.append(counter)
    percent_arr.append(percentage)

ga_dict = {'Gender' : store_g,
          'Count' : count_arr,
          'Percent' : percent_arr}

gd_df = pd.DataFrame(ga_dict)
gd_df

,Gender,Count,Percent
0,Male,484,0.84
1,Other / Non-Disclosed,11,0.02
2,Female,81,0.14


In [6]:
# Purchasing Analysis (Gender)

# The below each broken by gender

# Purchase Count
# Average Purchase Price
# Total Purchase Value
# Average Purchase Total per Person by Gender

pur_count = []
pur_price = []
avg_price = []

for i in range(len(unique_gender)):
    g = unique_gender[i]
    py_m = py_df[py_df['Gender']==g]
    g_count = py_m.count()
    pur_count.append(g_count[0])
    price = py_m['Price'].sum()
    avg = py_m['Price'].mean()
    pur_price.append(float(price))
    avg_price.append(float(round(avg,2)))

gender_dict = {'Gender': store_g,'Purchase Count':pur_count,'Total Purchase Value':pur_price,'Average Price':avg_price,
              'Unique Count': count_arr}
gender_pd = pd.DataFrame(gender_dict)
gender_pd['Pur By Person By Gen'] = round(gender_pd['Total Purchase Value']/gender_pd['Unique Count'],2)
gender_pd

,Gender,Purchase Count,Total Purchase Value,Average Price,Unique Count,Pur By Person By Gen
0,Male,652,1967.64,3.02,484,4.07
1,Other / Non-Disclosed,15,50.19,3.35,11,4.56
2,Female,113,361.94,3.20,81,4.47


In [7]:
## Get the bin count and percent

bins = [0, 9, 14, 19, 24, 29, 34, 39, 99]
names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

bins = pd.cut(dup_df['Age'], bins, labels=names)
bin_count = bins.count()
grouped = bins.groupby(bins)
count = bins.groupby(bins).count()

app_count = [] 
percent_app = []
for i in range(len(count)):
    x = (count[i])
    app_count.append(x)
    per = (x/bin_count)
    percent_app.append(per)

bin_dict = {'Bins': names, 'Count': app_count, 'Percent': percent_app}
bin_df = pd.DataFrame(bin_dict)
bin_df

,Bins,Count,Percent
0,<10,17,0.029514
1,10-14,22,0.038194
2,15-19,107,0.185764
3,20-24,258,0.447917
4,25-29,77,0.133681
5,30-34,52,0.090278
6,35-39,31,0.053819
7,40+,12,0.020833


In [8]:
## Get the stats needed for the bins

bins = [0, 9, 14, 19, 24, 29, 34, 39, 99]
names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

py_df.loc[:,'Age Group'] = pd.cut(py_df['Age'], bins=bins, labels=names)

gd_df = py_df.groupby('Age Group')
count = gd_df.count()
count = count['SN']
total_count = py_df.count()
avg_pur_price = gd_df['Price'].mean()
total_purchase_value = gd_df['Price'].sum()
avg_per_person = total_purchase_value/app_count

final_dict = {'Purchase Count': count, 'Average Purchase Price': round(avg_pur_price,2),
              'Total Purchase Value': round(total_purchase_value,2), 
              'Average Total Purchase Per Person': round(avg_per_person,2)}
final_pd = pd.DataFrame(final_dict)
final_pd

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase Per Person
Age Group,,,,
<10,23,3.35,77.13,4.54
10-14,28,2.96,82.78,3.76
15-19,136,3.04,412.89,3.86
20-24,365,3.05,1114.06,4.32
25-29,101,2.90,293.00,3.81
30-34,73,2.93,214.00,4.12
35-39,41,3.60,147.67,4.76
40+,13,2.94,38.24,3.19


In [9]:
## Top Spenders

sn_df = py_df.groupby('SN')
sn_count = sn_df['SN'].count()
avg_sn_count = sn_df['Price'].mean()
tot_sn = sn_df['Price'].sum()

sn_dict= {'Purchase Count': sn_count, 'Average Pur Price': avg_sn_count, 'Total Purchase Value': tot_sn}
final_sn_df = pd.DataFrame(sn_dict)
fn_sort = final_sn_df.sort_values(by='Total Purchase Value', ascending=False)
fn_sort.head()

,Purchase Count,Average Pur Price,Total Purchase Value
SN,,,
Lisosia93,5,3.792000,18.96
Idastidru52,4,3.862500,15.45
Chamjask73,3,4.610000,13.83
Iral74,4,3.405000,13.62
Iskadarya95,3,4.366667,13.10


In [10]:
## Most Popular Items

pi_df = py_df.groupby(['Item ID','Item Name'])
pi_count = pi_df['SN'].count()
pi_price = pi_df['Price'].max()

pi_dict = {'Purchase Count': pi_count, 'Price': pi_price}
final_df = pd.DataFrame(pi_dict)
final_df['Total Purchase Value'] = final_df['Purchase Count']*final_df['Price']
final_df_sort = final_df.sort_values(by='Purchase Count',ascending=False)
final_df_sort.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
82,Nirvana,9,4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16


In [11]:
## Most Profitable Item

pi_df = py_df.groupby(['Item ID','Item Name'])
pi_count = pi_df['SN'].count()
pi_price = pi_df['Price'].max()

pi_dict = {'Purchase Count': pi_count, 'Price': pi_price}
final_df = pd.DataFrame(pi_dict)
final_df['Total Purchase Value'] = final_df['Purchase Count']*final_df['Price']
final_df_sort = final_df.sort_values(by='Total Purchase Value',ascending=False)
final_df_sort.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80
